### Toy Corpus

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
corpus1 = [["je", "suis", "nicolas"], ["je", "suis", "enseignant"]]
corpus2 = [["je", "suis", "rémi"], ["je", "suis", "enseignant", "maintenant"]]

In [42]:
from sinr.text.cooccurrence import Cooccurrence
from sinr.text.pmi import pmi_filter
c = Cooccurrence()
c.fit(corpus1, window=10)
c.save("matrix1.pk")

In [43]:
import sinr.graph_embeddings as ge
sinr = ge.SINr.load_from_cooc_pkl("matrix1.pk")
communities = sinr.detect_communities(gamma=1)
sinr.extract_embeddings(communities)
sinr_vectors = ge.InterpretableWordsModelBuilder(sinr, "corpus1", n_jobs=8, n_neighbors=4).build()
print(sinr_vectors.get_communities_as_labels_sets())

2023-01-31 11:36:51,874 - load_from_cooc_pkl - INFO - Building Graph.
2023-01-31 11:36:51,876 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2023-01-31 11:36:51,877 - load_pkl_text - INFO - Finished loading data.
2023-01-31 11:36:51,877 - load_from_cooc_pkl - INFO - Finished building graph.
2023-01-31 11:36:51,878 - detect_communities - INFO - Detecting communities.
2023-01-31 11:36:51,879 - detect_communities - INFO - Finished detecting communities.
2023-01-31 11:36:51,880 - extract_embeddings - INFO - Extracting embeddings.
2023-01-31 11:36:51,880 - extract_embeddings - INFO - Applying NFM.
2023-01-31 11:36:51,880 - get_nfm_embeddings - INFO - Starting NFM
2023-01-31 11:36:51,881 - extract_embeddings - INFO - NFM successfully applied.
2023-01-31 11:36:51,882 - extract_embeddings - INFO - Finished extracting embeddings.


Communities detected in 0.00011 [s]
solution properties:
-------------------  -
# communities        1
min community size   4
max community size   4
avg. community size  4
imbalance            1
edge cut             0
edge cut (portion)   0
modularity           0
-------------------  -
[{'suis', 'nicolas', 'enseignant', 'je'}]


In [44]:
c = Cooccurrence()
c.fit(corpus2, window=10)
c.save("matrix2.pk")
import sinr.graph_embeddings as ge
sinr = ge.SINr.load_from_cooc_pkl("matrix2.pk")
#TRANSFERT IS DONE HERE
sinr.transfert_communities_labels(sinr_vectors.get_communities_as_labels_sets())
sinr.extract_embeddings()

2023-01-31 11:36:51,898 - load_from_cooc_pkl - INFO - Building Graph.
2023-01-31 11:36:51,899 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2023-01-31 11:36:51,899 - load_pkl_text - INFO - Finished loading data.
2023-01-31 11:36:51,900 - load_from_cooc_pkl - INFO - Finished building graph.
2023-01-31 11:36:51,900 - extract_embeddings - INFO - Extracting embeddings.
2023-01-31 11:36:51,900 - extract_embeddings - INFO - Applying NFM.
2023-01-31 11:36:51,900 - get_nfm_embeddings - INFO - Starting NFM
2023-01-31 11:36:51,902 - extract_embeddings - INFO - NFM successfully applied.
2023-01-31 11:36:51,902 - extract_embeddings - INFO - Finished extracting embeddings.


[4, 0, 1] 0
[4, 0, 1] 1


In [45]:
sinr_vectors = ge.InterpretableWordsModelBuilder(sinr, "corpus2", n_jobs=8, n_neighbors=4).build()

In [46]:
sinr_vectors.most_similar("maintenant")

{'object ': 'maintenant',
 'neighbors ': [('maintenant', 1.0), ('suis', 0.9), ('je', 0.9)]}

In [47]:
sinr_vectors.get_communities_as_labels_sets()

[{'maintenant'}, {'rémi'}, {'enseignant', 'je', 'suis'}]

### Reuters

In [67]:
# import nltk
# nltk.download('brown')
from sinr.text.cooccurrence import Cooccurrence
from sinr.text.pmi import pmi_filter
from nltk.corpus import brown
from nltk.corpus import reuters
c = Cooccurrence()
c.fit(reuters.sents()[0:len(reuters.sents()) - 1000], window=100)
c.matrix = pmi_filter(c.matrix)
c.save("matrix.pk")
import sinr.graph_embeddings as ge
sinr = ge.SINr.load_from_cooc_pkl("matrix.pk")
communities = sinr.detect_communities(gamma=300)

2023-01-31 12:57:44,376 - load_from_cooc_pkl - INFO - Building Graph.
2023-01-31 12:57:44,377 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2023-01-31 12:57:44,395 - load_pkl_text - INFO - Finished loading data.
2023-01-31 12:57:45,740 - load_from_cooc_pkl - INFO - Finished building graph.
2023-01-31 12:57:45,755 - detect_communities - INFO - Detecting communities.
/home/dugue/miniconda3/envs/py39/lib/python3.9/site-packages/networkit/stopwatch.py:49: UserWarning: networkit.Timer is deprecated, will be removed in future updates.
  warn("networkit.Timer is deprecated, will be removed in future updates.")
2023-01-31 12:57:46,419 - detect_communities - INFO - Finished detecting communities.


Communities detected in 0.59354 [s]
solution properties:
-------------------  --------------
# communities        9054
min community size      1
max community size     39
avg. community size     4.56505
imbalance               7.8
edge cut                2.47269e+07
edge cut (portion)      7.00815
modularity              0.00946038
-------------------  --------------


In [68]:
sinr.extract_embeddings(communities)
sinr_vectors = ge.InterpretableWordsModelBuilder(sinr, "reuters1", n_jobs=8, n_neighbors=15).build()
sinr_vectors.most_similar("liberal")

2023-01-31 12:57:46,434 - extract_embeddings - INFO - Extracting embeddings.
2023-01-31 12:57:46,435 - extract_embeddings - INFO - Applying NFM.
2023-01-31 12:57:46,436 - get_nfm_embeddings - INFO - Starting NFM
/home/dugue/Depotgit/sinr/sinr/nfm.py:40: RuntimeWarning: divide by zero encountered in reciprocal
  np.reciprocal(community_weights).astype('float'))  # 1/community_weight for each column of the membership matrix
2023-01-31 12:57:57,327 - extract_embeddings - INFO - NFM successfully applied.
2023-01-31 12:57:57,327 - extract_embeddings - INFO - Finished extracting embeddings.


{'object ': 'liberal',
 'neighbors ': [('Republicans', 0.67),
  ('revolt', 0.66),
  ('defeat', 0.65),
  ('alliance', 0.65),
  ('legislators', 0.63),
  ('winning', 0.62),
  ('Democrats', 0.62),
  ('conservative', 0.59),
  ('statement', 0.55),
  ('sharp', 0.51),
  ('drop', 0.5),
  ('speech', 0.5),
  ('slight', 0.5),
  ('minute', 0.49)]}

In [69]:
sinr_vectors.get_dimension_descriptors("liberal")

{'dimension': 4415, 'descriptors': [(0.12, 'revolt'), (0.1, 'alliance'), (0.09, 'defeat'), (0.08, 'legislators'), (0.08, 'Republicans'), (0.07, 'liberal'), (0.06, 'winning'), (0.05, 'Democrats'), (0.04, 'conservative'), (0.04, 'minute')]}

In [70]:
sinr_vectors.get_number_of_dimensions()

9054

In [71]:
c = Cooccurrence()
c.fit(reuters.sents()[1000:], window=10)
c.save("matrix2.pk")
import sinr.graph_embeddings as ge
sinr = ge.SINr.load_from_cooc_pkl("matrix2.pk")
sinr.transfert_communities_labels(sinr_vectors.get_communities_as_labels_sets())
sinr.extract_embeddings()
sinr_vectors = ge.InterpretableWordsModelBuilder(sinr, "reuters2", n_jobs=8, n_neighbors=15).build()
sinr_vectors.most_similar("liberal")

2023-01-31 12:58:05,876 - load_from_cooc_pkl - INFO - Building Graph.
2023-01-31 12:58:05,877 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2023-01-31 12:58:05,890 - load_pkl_text - INFO - Finished loading data.
2023-01-31 12:58:07,084 - load_from_cooc_pkl - INFO - Finished building graph.
2023-01-31 12:58:07,126 - extract_embeddings - INFO - Extracting embeddings.
2023-01-31 12:58:07,127 - extract_embeddings - INFO - Applying NFM.
2023-01-31 12:58:07,128 - get_nfm_embeddings - INFO - Starting NFM
2023-01-31 12:58:16,845 - extract_embeddings - INFO - NFM successfully applied.
2023-01-31 12:58:16,845 - extract_embeddings - INFO - Finished extracting embeddings.


{'object ': 'liberal',
 'neighbors ': [('conservatives', 0.63),
  ('Democrats', 0.62),
  ('alliance', 0.6),
  ('economy', 0.58),
  ('growing', 0.58),
  ('pact', 0.58),
  ('to', 0.58),
  ('dollar', 0.58),
  ('industry', 0.58),
  ('bill', 0.57),
  ('Republicans', 0.57),
  ('market', 0.57),
  ('made', 0.57),
  ('proposed', 0.57)]}

In [72]:
sinr_vectors.get_dimension_descriptors("liberal")

{'dimension': 6139, 'descriptors': [(0.1, 'defeat'), (0.1, 'alliance'), (0.1, 'revolt'), (0.06, 'Republicans'), (0.05, 'liberal'), (0.04, 'Democrats'), (0.04, 'minute'), (0.03, 'conservative'), (0.02, 'legislators'), (0.0, 'winning')]}

In [73]:
sinr_vectors.get_number_of_dimensions()

9309